image    
[food] : 5000      
[interior] : 5000    
[exterior] : 5000    
      
작은 양의 데이터로 일단 predict 해보기.  

활성함수   
1) Conv2D(32, kernel_size=3, activation='relu)  
2) Dropout(0.5)   
3) MaxPooling2D(pool_size=2)  
  
4) Conv2D(64, kernel_size=3, activation='relu')  
5) MaxPooling2D(pool_size=2)  
  
6) Dense(10,activation='selu')  
7) Flatten()  
  
----
- Acc : 0.63]
- val_acc : 0.397
- predict : 0.4736

In [2]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

from keras.models import Sequential,load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)

image  
[food] : 5000    
[interior] : 5000  
[exterior] : 5000  

In [2]:
def image_load_data():
    train_images = []       
    train_labels = []
    shape = (300,300)  
    train_path = './image'

    for classes in ['f','i','e']:
        for filename in os.listdir('./image'):
            if classes == filename[0]:
                img = cv2.imread(os.path.join(train_path,filename))
                #print(filename)
                # print(img)
                # Spliting file names and storing the labels for image in list
                train_labels.append(filename[2:0:-1])

                # Resize all images to a specific shape
                #img = cv2.resize(img,shape)

                train_images.append(img)
            else:
                pass

    # Converting labels into One Hot encoded sparse matrix

    # 음식, 실내 ,실외 순으로 들어가기 위해 index 처리
    # train_labels = sorted(train_labels)
    # print(train_labels)

    train_labels = pd.get_dummies((train_labels)).values
    # print(train_labels)
    # Converting train_images to array
    train_images = np.array(train_images)

    # Splitting Training data into train and validation dataset

    train_images ,train_labels= shuffle(train_images ,train_labels, random_state = 42) 

    x_train,x_test,y_train,y_test = train_test_split(train_images,train_labels,test_size=0.2,random_state=42)
    # print(train_labels.shape)
    # print(train_labels)
    # print(len(x_train),len(x_test),len(y_train),len(y_test))

    return x_train, x_test, y_train, y_test 


In [11]:
def plot_loss_curve(history):
    plt.figure(figsize = (5,3))
    
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['train','test'], loc = 'upper right')
    plt.show

활성함수  
1) Conv2D(32, kernel_size=3, activation='relu)  
2) Dropout(0.5)   
3) MaxPooling2D(pool_size=2)   
  
4) Conv2D(64, kernel_size=3, activation='relu')  
5) MaxPooling2D(pool_size=2)    
  
6) Dense(10,activation='selu')   
7) Flatten()   

In [12]:
def train_cnn_model():
# Creating a Sequential model
    
    model = Sequential([ Input(shape=(300,300,3), name='input_layer'),
        Conv2D(32, kernel_size=3, activation='relu', name='conv_layer1'),

        Dropout(0.5),
        MaxPooling2D(pool_size=2),
        Conv2D(64, kernel_size=3, activation='relu', name='conv_layer2'),
        MaxPooling2D(pool_size=2),
        #Dense(20, activation='softmax', name='output_layer')
        Dense(10,activation='selu',name='FC_layer1'),
        Flatten(),
        Dense(3, activation='softmax', name='output_layer')
            ])

    model.compile(optimizer= 'adam', loss='categorical_crossentropy',metrics=['accuracy'])

    # Model Summary
    model.summary()

    # Training the model
    history = model.fit(X_train,y_train, validation_data=(X_test, y_test), batch_size=50, epochs = 3)
    model.save('model-2020-1215-12시')
    plot_loss_curve(history.history)
    print(history.history)
    print("train loss=", history.history['loss'][-1])
    print("validation loss=", history.history['val_loss'][-1])
    return model

In [8]:
X_train,X_test,y_train, y_test =image_load_data()
model, history = train_cnn_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_layer1 (Conv2D)         (None, 298, 298, 32)      896       
_________________________________________________________________
dropout (Dropout)            (None, 298, 298, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv_layer2 (Conv2D)         (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
FC_layer1 (Dense)            (None, 73, 73, 10)        650       
_________________________________________________________________
flatten (Flatten)            (None, 53290)             0

In [81]:
def predict_image_sample(model,X_test, y_test, test_id= -1 ):
    if test_id <0 :
        from random import randrange
        test_sample_id = randrange(10000)
    else:
        test_sample_id = test_id
    test_image =X_test[test_sample_id].reshape(1,300,300,3)
#     test_image =test_images[test_sample_id]
#     plt.imshow(test_images[123])

    y_actual = y_test[test_sample_id].tolist().index(1)
    print('y_actual number = ', y_actual)
    
    y_pred = model.predict(test_image)
    y_pred = np.round(y_pred,6)
#     np.set_printoptions(suppress=True)
    print("y_pred = ", y_pred)

    y_pred_num = np.argmax(y_pred, axis = 1)[0]
    print('y_pred number = ', y_pred_num)
    return y_actual , y_pred_num


In [18]:
if __name__ == '__main__' :
    n = 0
    model = load_model('model-2020-1215-12시')
    X_train,X_test,y_train, y_test =image_load_data()
    for i in range(20):
        y_actual , y_pred_num = predict_image_sample(model,X_test, y_test,test_id = i)
        
        if y_actual == y_pred_num:
            n += 1 
        else : pass
print('전체 횟수 :', i,' '*10,'예측 성공 횟수',n,'\n'*2 ,'정확도 = ', (n/i)*100)

y_actual number =  1
y_pred =  [[4.4322405e-08 3.3770699e-05 9.9996614e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.94136703 0.03071299 0.02791999]]
y_pred number =  0
y_actual number =  1
y_pred =  [[2.8446900e-26 2.4888816e-10 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[4.8659822e-05 3.6083180e-02 9.6386814e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[5.5181722e-08 5.6241413e-05 9.9994373e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.06228878 0.9307277  0.00698346]]
y_pred number =  1
y_actual number =  1
y_pred =  [[1.4830058e-20 2.7694616e-13 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[7.5816274e-27 2.8247847e-13 1.0000000e+00]]
y_pred number =  2
y_actual number =  1
y_pred =  [[6.2382595e-20 1.1661422e-09 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[1.61301296e-06 1.13165475e-01 8.86832952e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[1.6093643e-27 1.914483

In [21]:
n = 0
for i in range(1000):
        y_actual , y_pred_num = predict_image_sample(model,X_test, y_test,test_id = i)
        
        if y_actual == y_pred_num:
            n += 1 
        else : pass
print('전체 횟수 :', i,' '*10,'예측 성공 횟수',n,'\n'*2 ,'정확도 = ', (n/i)*100)

y_actual number =  1
y_pred =  [[4.4322405e-08 3.3770699e-05 9.9996614e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.94136703 0.03071299 0.02791999]]
y_pred number =  0
y_actual number =  1
y_pred =  [[2.8446900e-26 2.4888816e-10 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[4.8659822e-05 3.6083180e-02 9.6386814e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[5.5181722e-08 5.6241413e-05 9.9994373e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.06228878 0.9307277  0.00698346]]
y_pred number =  1
y_actual number =  1
y_pred =  [[1.4830058e-20 2.7694616e-13 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[7.5816274e-27 2.8247847e-13 1.0000000e+00]]
y_pred number =  2
y_actual number =  1
y_pred =  [[6.2382595e-20 1.1661422e-09 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[1.61301296e-06 1.13165475e-01 8.86832952e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[1.6093643e-27 1.914483

y_pred =  [[2.9008863e-14 3.1299249e-10 1.0000000e+00]]
y_pred number =  2
y_actual number =  1
y_pred =  [[1.6131767e-13 5.2457303e-01 4.7542691e-01]]
y_pred number =  1
y_actual number =  1
y_pred =  [[1.5924821e-30 2.6189564e-08 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.0031605  0.23253219 0.7643074 ]]
y_pred number =  2
y_actual number =  1
y_pred =  [[2.4908195e-13 2.1178079e-05 9.9997878e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[4.1743015e-06 2.0300955e-04 9.9979287e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.51313007 0.01681727 0.47005266]]
y_pred number =  0
y_actual number =  0
y_pred =  [[1.9567041e-03 6.9256395e-04 9.9735069e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[1.2971918e-20 1.3199077e-10 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[4.5389447e-24 2.8225771e-09 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[1.5889538e-14 3.2128182e-07 9.9999964e-01]]
y_

y_pred =  [[0.5634622  0.01822012 0.41831765]]
y_pred number =  0
y_actual number =  2
y_pred =  [[3.8145907e-29 1.5472361e-11 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[1.5718337e-09 3.8860858e-06 9.9999607e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.01832859 0.00923365 0.9724378 ]]
y_pred number =  2
y_actual number =  1
y_pred =  [[1.0443298e-22 6.2345609e-04 9.9937660e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[1.0472921e-05 9.7202414e-05 9.9989235e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[9.9531020e-18 1.0421626e-07 9.9999988e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[8.454095e-23 3.444123e-08 1.000000e+00]]
y_pred number =  2
y_actual number =  1
y_pred =  [[1.19085286e-07 7.18357088e-03 9.92816329e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[2.8689496e-17 4.6580544e-07 9.9999952e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[6.9974151e-11 5.6050754e-05 9.9994397e-01]]
y_

y_pred =  [[6.8030226e-12 4.5047139e-04 9.9954945e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[5.5530020e-18 4.7652724e-13 1.0000000e+00]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.00102736 0.01558604 0.9833866 ]]
y_pred number =  2
y_actual number =  2
y_pred =  [[2.7120322e-07 7.3558703e-04 9.9926418e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[4.8934259e-26 6.2056493e-12 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[2.6824464e-05 6.2844333e-06 9.9996686e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[4.2673298e-16 7.2228512e-10 1.0000000e+00]]
y_pred number =  2
y_actual number =  1
y_pred =  [[4.8856120e-18 1.1713815e-08 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[3.8375080e-02 6.6011224e-04 9.6096486e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[2.7257323e-07 2.9745343e-04 9.9970227e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[6.8195493e-12 8.0437399e-05 9.9991953

y_pred =  [[2.9509186e-12 2.1719393e-07 9.9999976e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[5.2768468e-05 4.7192473e-02 9.5275474e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[2.2499046e-19 3.1479314e-07 9.9999964e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[2.9179817e-01 7.6975512e-06 7.0819408e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[5.425519e-28 9.921471e-10 1.000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[9.7731189e-24 3.0621447e-08 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[1.1761541e-15 1.6078634e-04 9.9983919e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[1.4239769e-14 1.8775010e-05 9.9998128e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[1.9427546e-17 2.9405020e-04 9.9970597e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[3.5809811e-26 2.6065673e-12 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[3.6020834e-26 2.8032212e-06 9.9

...

y_pred =  [[0.16675115 0.00180853 0.8314404 ]]
y_pred number =  2
y_actual number =  2
y_pred =  [[2.1630792e-14 2.0967188e-04 9.9979037e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[4.0241122e-17 8.5776890e-05 9.9991417e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[1.2736306e-23 2.8607978e-06 9.9999714e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[1.3152983e-07 5.4621296e-06 9.9999440e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.04823907 0.92994285 0.02181808]]
y_pred number =  1
y_actual number =  2
y_pred =  [[1.4349730e-28 1.4219925e-10 1.0000000e+00]]
y_pred number =  2
y_actual number =  1
y_pred =  [[2.3614286e-11 1.2169903e-04 9.9987829e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[5.5914948e-06 9.0591551e-05 9.9990380e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[5.1687734e-13 8.9714922e-07 9.9999905e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[7.816825e-17 6.300607e-06 9.999937e-01]]
y_pre

y_pred =  [[4.9876036e-07 1.0185764e-05 9.9998927e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[1.4835001e-07 1.4048702e-01 8.5951281e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[3.7285101e-06 4.9174909e-04 9.9950457e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[3.7018612e-08 8.2966039e-12 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[8.1675971e-05 1.7609142e-01 8.2382697e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[2.1838503e-23 6.2269467e-11 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[2.0720986e-14 2.6371723e-08 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[1.8226180e-21 1.2012242e-09 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[9.997315e-01 1.756934e-04 9.285272e-05]]
y_pred number =  0
y_actual number =  1
y_pred =  [[8.8372608e-06 7.3462096e-04 9.9925655e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[2.3852833e-06 4.1633857e-06 9.9

y_pred =  [[8.259970e-35 4.323221e-15 1.000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[1.4192513e-35 3.5676914e-16 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[7.961688e-06 3.600465e-04 9.996320e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[3.8169826e-34 6.9206544e-13 1.0000000e+00]]
y_pred number =  2
y_actual number =  1
y_pred =  [[5.6257891e-15 1.8169405e-04 9.9981838e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[5.5302658e-22 3.1283342e-12 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[2.1863588e-31 3.1428044e-10 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.3425809  0.02992611 0.62749296]]
y_pred number =  2
y_actual number =  1
y_pred =  [[4.5980188e-16 1.2078159e-06 9.9999881e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[5.1541064e-14 6.0242165e-08 9.9999988e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[3.0260873e-05 2.2810095e-06 9.9996746e-01]]

y_pred =  [[3.3323372e-12 2.6930541e-02 9.7306943e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[6.2170136e-07 6.6162139e-02 9.3383729e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[2.6191996e-13 1.2638577e-08 1.0000000e+00]]
y_pred number =  2
y_actual number =  2
y_pred =  [[1.7622066e-10 2.8351553e-02 9.7164840e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[6.170139e-19 9.121006e-06 9.999908e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[3.5075893e-04 3.4274894e-06 9.9964583e-01]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.5179413  0.00124726 0.4808115 ]]
y_pred number =  0
y_actual number =  1
y_pred =  [[3.3101924e-10 9.4257761e-03 9.9057418e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[6.081138e-10 7.477888e-06 9.999925e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[2.3096759e-19 2.2327544e-05 9.9997771e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[8.1436455e-12 2.0081103e-07 9.9999976e-01]]

y_pred =  [[2.0108622e-12 1.5853175e-05 9.9998415e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[4.9265984e-14 5.5447879e-05 9.9994457e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[6.0084376e-30 2.4408973e-06 9.9999762e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[4.2280458e-19 4.9859139e-08 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[3.1095002e-08 2.7268369e-02 9.7273165e-01]]
y_pred number =  2
y_actual number =  1
y_pred =  [[4.4726679e-19 2.4891915e-06 9.9999750e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[3.5953778e-35 1.0836544e-12 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.14167452 0.8460599  0.01226553]]
y_pred number =  1
y_actual number =  1
y_pred =  [[2.6812560e-20 1.3444275e-05 9.9998653e-01]]
y_pred number =  2
y_actual number =  2
y_pred =  [[7.0236507e-16 1.5490714e-09 1.0000000e+00]]
y_pred number =  2
y_actual number =  0
y_pred =  [[1.7554823e-03 4.2404612e-05 9.9820209

In [86]:
n = 0
for i in range(50):
        y_actual , y_pred_num = predict_image_sample(model,X_test, y_test,test_id = i)
        if y_actual == y_pred_num:
            n += 1 
        else : pass
print('전체 횟수 :', i,' '*10,'예측 성공 횟수',n,'\n'*2 ,'정확도 = ', (n/i)*100)

y_actual number =  1
y_pred =  [[0.       0.000034 0.999966]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.941367 0.030713 0.02792 ]]
y_pred number =  0
y_actual number =  1
y_pred =  [[0. 0. 1.]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000049 0.036083 0.963868]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.       0.000056 0.999944]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.062289 0.930728 0.006983]]
y_pred number =  1
y_actual number =  1
y_pred =  [[0. 0. 1.]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0. 0. 1.]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0. 0. 1.]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000002 0.113165 0.886833]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0. 0. 1.]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0.       0.324361 0.675639]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000028 0.000143 0.999829]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0. 0

In [84]:
print(X_train.shape, '\n',X_test.shape, '\n',y_train.shape, '\n',y_test.shape)

(12000, 300, 300, 3) 
 (3000, 300, 300, 3) 
 (12000, 3) 
 (3000, 3)


In [82]:
n = 0
for i in range(6,15):
        y_actual , y_pred_num = predict_image_sample(model,X_test, y_test,test_id = i)
        
        if y_actual == y_pred_num:
            n += 1 
        else : pass
print('전체 횟수 :', 15-6,' '*10,'예측 성공 횟수',n,'\n'*2 ,'정확도 = ', (n/i)*100)

y_actual number =  1
y_pred =  [[0. 0. 1.]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0. 0. 1.]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0. 0. 1.]]
y_pred number =  2
y_actual number =  0
y_pred =  [[0.000002 0.113165 0.886833]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0. 0. 1.]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0.       0.324361 0.675639]]
y_pred number =  2
y_actual number =  1
y_pred =  [[0.000028 0.000143 0.999829]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0. 0. 1.]]
y_pred number =  2
y_actual number =  2
y_pred =  [[0. 0. 1.]]
y_pred number =  2
전체 횟수 : 9            예측 성공 횟수 4 

 정확도 =  28.57142857142857
